In [35]:
from aes_stream import AES_ECB, AES_CBC, AES_CFB, AES_OFB, AES_CTR
from rc4 import RC4
from salsa20 import Salsa20
import numpy as np
import time
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

sns.set(style="whitegrid")

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
def test_time(Algo, n_bytes_key, n_bytes_stream):
    key = np.random.randint(0, 256, n_bytes_key, dtype=np.uint8)
    stream = np.random.randint(0, 256, n_bytes_stream, dtype=np.uint8)
    
    start = time.time()
    cipher = Algo(key)
    assert np.array_equal(stream, cipher.decode(cipher.encode(stream))), str(Algo) + " works bad!"
    
    return time.time() - start

In [12]:
test_time(RC4, 256, 512)

0.014789342880249023

In [13]:
test_time(Salsa20, 32, 512)

0.037580251693725586

In [28]:
test_time(AES_ECB, 16, 512)

0.39632344245910645

In [15]:
test_time(AES_CBC, 16, 512)

0.388901948928833

In [32]:
test_time(AES_CFB, 16, 512)

1.4803411960601807

In [27]:
test_time(AES_OFB, 16, 512)

0.028011560440063477

In [23]:
test_time(AES_CTR, 16, 512)

0.37130260467529297

In [ ]:
data = {"RC4": [], "Salsa20": [], "AES_ECB": [], "AES_CBC": [], "AES_CFB": [], "AES_OFB": [], "AES_CTR": []}
points = 10
init_power = 9
power_bytes = init_power
upper_bound = 30
rc4_s = salsa20_s = ecb_s = cbc_s = cfb_s = ofb_s = ctr_s = 0
for i in range(points):
    print(i)
    n_bytes = 2**power_bytes
    
    if rc4_s < upper_bound:
        rc4_s = test_time(RC4, 256, n_bytes)

    if salsa20_s < upper_bound:
        salsa20_s = test_time(Salsa20, 32, n_bytes)
    
    if ecb_s < upper_bound:
        ecb_s = test_time(AES_ECB, 16, n_bytes)
        
    if cbc_s < upper_bound:
        cbc_s = test_time(AES_CBC, 16, n_bytes)
        
    if cfb_s < upper_bound:
        cfb_s = test_time(AES_CFB, 16, n_bytes)
        
    if ofb_s < upper_bound:
        ofb_s = test_time(AES_OFB, 16, n_bytes)
    
    if ctr_s < upper_bound:
        ctr_s = test_time(AES_CTR, 16, n_bytes)
    
    data["RC4"].append(rc4_s)
    data["Salsa20"].append(salsa20_s)
    data["AES_ECB"].append(ecb_s)
    data["AES_CBC"].append(cbc_s)
    data["AES_CFB"].append(cfb_s)
    data["AES_OFB"].append(ofb_s)
    data["AES_CTR"].append(ctr_s)
    
    power_bytes += 1

0
1
2
3
4
5
6
7


In [ ]:
plt.tight_layout()
plt.rcParams["figure.figsize"] = (13,10)

In [ ]:
for k in data:
    y = data[k]
    x = list(range(len(y)))
    
    ax = sns.lineplot(x=x, y=y, linewidth=2.5, label=k)

plt.xticks(range(points))
ax.set_xticklabels([f"2^{i}" for i in range(init_power, init_power + points)])
ax.set(ylabel="Seconds", xlabel="Bytes in stream")